# The Golden Age of Apocalyptic Literature
### Or How I Learned to Stop Worrying and Love the End
##### by Duncan Gibbs

## Introduction

I find myself thinking about the end of the world a lot recently.

From the balcony of my apartment - where I've been quarantined since March due to a sweeping global pandemic - things look bleak. I can see the rise of right-wing authoritarianism and nationalism around the world, as well as decades of anti-environmental policies come back to haunt us. For genuinely uncountable reasons (whose existence in list form would serve only to depress me), 2020 has felt like the worst year in a long time, if not my lifetime.

Stick with me for a second; I used to travel a lot for a job I had. The fascinating thing that happens when you fly across the country every two weeks is that travel stops meaning anything to you. Over preparation, nervousness, and arriving hours early to the airport were slowly and imperceptibly supplanted by minimalism, casualness, and sleeping in as late as I could. One Sunday night - before I flew to Sterling, VA on Monday morning - I decided that enough was enough, and I had to clean out my fridge. I threw away three moldy tomatoes, a bag of liquefied and browning spinach, expired condiments, and some jars and boxes with no detectable date on it. The next morning, without a care in the world, I left for Virginia. 10 days later, I was sitting in my car in the parking garage underneath my apartment. I had remembered as I pulled in that I left a trash can full of rotten food in my kitchen. The next 30 minutes were a silent montage of me convincing myself that the smell was going to be unbearable. That the smell was never going to come out, and not only would I have to live with it for the rest of my lease, but I was going to lose my security deposit. That I was going to suddenly be known as "the smelly kid." That upon smelling the odor emanating from my apartment, a helpful neighbor had called the police to report a dead body. That the police were up there waiting for me now to evict me or take me off to jail. That they had already been consumed by the moldy tomato monster my mistake had born.

The lesson I would learn from this was internalized by my discovery of only a smelly trashcan waiting for me: anxiety is not based on reality.

With the brief section where I offhandedly mention that I suffer from anxiety and am prone to catastrophizing at possibly an olympic level out of the way, we can continue.

So the question I had to ask myself was, "Am I catastrophizing how close the world is to ending?" Or asked in a possibly more relatable way, "Is this feeling in the pit of my stomach just recency bias and anxiety?" And despite the fact that the Doomsday Clock was moved to 100 seconds from midnight this year in January (before Coronavirus or anything else on the untypeable list), I didn't think that was a question anyone could actually answer. If I had to guess, that's a question only survivors of a horrible nuclear winter will be able to ask and answer amongst temselves around the dying embers of their last camp fire. When a question seems unanswerable though, the only option is to reframe it. And when cold, hard facts provide no comfort to me, I instead search for human connection.

Unfortunately, the next question I asked was equally unsolvable. "Has society ever been this worried about the end of the world before?" Now while that may not be a totally unsolvable question, it's way beyond the scope of anything I could personally do in a couple of weekends. Or maybe ever. So instead I had to make the question a little more specific.

And so we come to the title: "What is the Golden Age of apocalyptic literature?" At the outset this seemed like a question I had the actual personal ability to answer. But while working on this I realized it was not actually the question I wanted to answer. Despite being the next 10,810 words, 17 graphs, and 8 tables worth of discussion, the actual hypothesis I will investigate is this: 

The people and societies of the past - as reflected by their fictional literature - have been more worried about an impending apocalypse than I currently feel.

## The Data

Before we can get into the juicy details though, we need to talk about how I acquired the data I will be using. Which means talking about both Goodreads as a data source, as well as the inherent intractability of genre, publishing, and book cataloging in general.

### Tools Used

---

**The "Tools Used" section is the one I would recommend skipping if you are neither a programmer nor interested in the nitty gritty of what I've built. But also it's pretty short and I'm not the boss of you.**

---

The page you are viewing now is either an interactive Jupyter Notebook or was generated from one. With almost no evidence to back it up, I assume Python is the most popular language to write Jupyter Notebooks in. Which makes sense considering the command line interactivity of Python, the tooling available, and the community around it. Unfortunately - and despite what my resume says - I hate Python. Luckily [evcxr](https://github.com/google/evcxr) exists. So the scraper and notebook are all written in Rust.

For scraping the data from Goodreads, I found that their API was lacking a lot of the data that would show up on their site. Most notably [book genres](https://www.goodreads.com/topic/show/19317219-getting-the-genre-of-a-book). Clearly a lot of what they display on the website is only available on the backend... Or the frontend if you're willing to fake it. I used [the fantoccini crate](https://docs.rs/fantoccini/0.14.1/fantoccini/index.html) and [ChromeDriver](https://chromedriver.chromium.org/) to log in to Goodreads and pull all of the information for specific books.

The other tool I used is [the plotters crate](https://docs.rs/plotters/0.3.0/plotters/index.html) to generate all of the charts and tables you will see. Luckily it has built in support for evcxr and jupyter.

Lastly, [this project](https://github.com/duncgibbs/apocalyptic-literature-stats) is not limited to the data I present here. At that link, I have built the generic ability to scrape certain information from all books in any genre or get the total number of books in a genre from Goodreads (those are surprisingly different things). I am currently storing that data as JSON in the project. As of the writing of this, only the data I needed is present, but I look forward to the data possibly being expanded.

In [319]:
:dep als = {path = ".", package = "apocalyptic-literature-stats"}
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

extern crate plotters;
use plotters::prelude::*;

### Goodreads

The problem with Goodreads as a source of truth for books is most easily explained by showing you this chart:

In [320]:
evcxr_figure((1000, 300), |root| {
    let books_per_year = als::viz::get_books_per_year_percent_partial("apocalyptic")?;
    let mut danger_years: Vec<i32> = (2010..2016).collect();
    
    let mut current_pixel = 0;
    let mut idx = 0;
    let mut danger_year_pixels = (0, 0);
    
    for (year, percent) in books_per_year {
        if percent > 0.0 {
            let year_width = (percent * 1000.0).round() as i32;
            root.draw(
                &Rectangle::new(
                    [(current_pixel, 100), (current_pixel + year_width, 300)],
                    Into::<ShapeStyle>::into(&Palette99::pick(idx)).filled()
                )
            )?;
            
            if danger_years[0] == year {
                danger_year_pixels.0 = current_pixel;
            }
            
            current_pixel += year_width;
            
            if *danger_years.last().unwrap() == year {
                danger_year_pixels.1 = current_pixel;
            }
            
            idx += 1;
        }
    }
    
    root.draw(
        &Rectangle::new(
            [(danger_year_pixels.0, 1), (danger_year_pixels.1, 300)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    root.draw(
        &Text::new(
            "2010 - 2016",
            (danger_year_pixels.0 + 240, 40),
            ("Arial", 32).into_font()
        )
    )?;

    Ok(())
})

2010 - 2016

Each colored stripe along this bar represents a year. The width of the stripe represents the amount of our data that comes from that year.

The six year span from 2010-2016 represents roughly 59.2% of the data. Things get even worse when you add up the last twenty years and realize the 21st century accounts for 84.7% of apocalyptic fiction on Goodreads. This makes sense as quickly comparing the popularity of Goodreads on Google Trends against the books per year shows us a pattern.

In [321]:
evcxr_figure((800, 600), |root| {
    let popularity_color = Palette99::pick(0);
    let books_color = Palette99::pick(1);
    let areas = root.split_evenly((2,1));

    let trends = als::viz::get_goodreads_google_trends()?;
    let books = als::viz::get_books_per_year_vec("apocalyptic")?;
    
    let mut trends_chart = ChartBuilder::on(&areas[0])
        .caption("Popularity of Goodreads on Google Trends", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(2004..2021, 0..1300)?;

    trends_chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_labels(10)
        .y_labels(5)
        .x_desc("Year")
        .y_desc("Popularity")
        .draw()?;

    let trends_hist = Histogram::vertical(&trends_chart)
        .style(popularity_color.filled())
        .margin(1)
        .data(trends);
    
    trends_chart.draw_series(trends_hist)?;
    
    let mut books_chart = ChartBuilder::on(&areas[1])
        .caption("Apocalyptic Books on Goodreads", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(2004..2021, 0..200)?;

    books_chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_labels(10)
        .y_labels(5)
        .x_desc("Year")
        .y_desc("Books")
        .draw()?;

    let books_hist = Histogram::vertical(&trends_chart)
        .style(books_color.filled().filled())
        .margin(1)
        .data(books);
    
    books_chart.draw_series(books_hist)?;

    Ok(())
})

Popularity of Goodreads on Google Trends
 
 
Popularity
 
 
Year
 
 
 
0
 
 
 
500
 
 
 
1000
 
 
 
 
2004
 
 
 
2006
 
 
 
2008
 
 
 
2010
 
 
 
2012
 
 
 
2014
 
 
 
2016
 
 
 
2018
 
 
 
2020
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Apocalyptic Books on Goodreads
 
 
Books
 
 
Year
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
 
2004
 
 
 
2006
 
 
 
2008
 
 
 
2010
 
 
 
2012
 
 
 
2014
 
 
 
2016
 
 
 
2018
 
 
 
2020

Saying Goodreads has a "recency bias" is a bit like saying Jeff Bezos has "a lot of money." Not coincidentally, Goodreads was acquired by Amazon in 2013. I imagine some business analysts were looking at charts that beared more than a passing resemblance to these. Also, I promise to only bring this up this one time, but Jeff Bezos - the personification of making the wrong choice on the Trolley Problem - can go fuck himself.

The major way this recency bias will rear its head is by not only having more data - and more accurate data - recently but also by coloring all historical data with a modern lens. Goodreads wasn't around in 1949, so when we later look at the user ratings of George Orwell's _1984_, what we're actually looking at is how people from 2012-2015 rated _1984_.

Another problem with the data is how "genres" work on Goodreads. The quotes around that word are entirely necessary because Goodreads doesn't actually have "genres." Instead, users place books on differently named shelves, which can include clear genres like "fiction" and "nonfiction" as well as the classics such as "science fiction" and "romance." But this also includes shelves like "currently reading" and "my favorite books." This makes genres/shelves on Goodreads little more than user submitted tags. So when getting books from the "apocalyptic" genre, we are relying on users of Goodreads to determine what is and is not apocalyptic fiction. But given the awe inspiring mess that is genre theory, this may be a blessing in disguise.

There are some other problems with the Goodreads data that we will discuss more later. But an even more fundamental problem awaits us.

### Intractability

If you only take away a single fact from this entire project, let it be this: we have no idea how many books there are. On the surface, a question like "How many books are there?" seems unanswerable, right? Who knows how many books were published in ye olden times, let alone modern books that are independently published or violently shoved onto the growing eBook store market. That wasn't what surprised me. What surprised me was that no amount of narrowing that question brought it anywhere nearer to answerable. Even something like "How many English language books have been published by a major publishing house?" is [a question without an answer](https://litlab.stanford.edu/how-many-novels-have-been-published-in-english-an-attempt/) (a fascinating article, by the way).

This isn't a huge deal since I'm not actually attempting any sort of real statistics or science with this. In order to do that, I'd have to design something far broader in scope. For Goodreads data, I'd have to grab a random sample of books and then look at the subset of apocalyptic books within that set. And I have already spent too much time on a thing that 12 people will see, and 5 will read until the end. So we'll just move right along.

### Outliers

While I will not be engaging with the forbidden art of statistics, I will at least be mimicking its facade. This will involve removing some bits of data. Most notably, all books before 1945. People have always been able to imagine an end to everything. Apocalyptic literature has been a genre of fiction since literature itself. The Epic of Gilgamesh, a 4000 year old epic poem, contains passages of world ending destruction. And in fact, many of the books we think of as quintessentially apocalyptic are from pre-1900. But boo hoo to those books. Not only do I think that everyone who wrote about the end of the world before the invention of the atomic bomb was just being a baby about death with metaphor and flowery language, but also it makes my data, work, and conclusion easier. Before moving on entirely though, I want to list what we are not including in the data going forward and give some shout outs to the OGs of imagined doomsdays.

In [322]:
{
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| {
        return book.year_published < 1945 && book.year_published != 0;
    });
    books.sort_by(|book_one, book_two| {
       return book_one.year_published.cmp(&book_two.year_published) ;
    });
    
    als::viz::print_books(books);
}

Title,Author,Year Published
The Last Man,Mary Wollstonecraft Shelley,1826
"After London: or, Wild England",Richard Jefferies,1885
The Time Machine,H.G. Wells,1895
The War of the Worlds,H.G. Wells,1898
The Purple Cloud,Matthew Phipps Shiel,1901
The Machine Stops,E.M. Forster,1909
The Scarlet Plague,Jack London,1912
Brave New World,Aldous Huxley,1932
"When Worlds Collide (When Worlds Collide, #1)",Philip Wylie,1932
Anthem,Ayn Rand,1938


Not only are some of these apocalypses truly archetypical, but many of these authors were the best to ever do it. Ayn Rand is also on this list.

The real contest in my mind is: which of these books is the most prophetic? The obvious - and probably correct - choice is _Brave New World_ by Aldolus Huxley. I mean, it's absolutely spooky what he continues to have understood about humanity. But I think in 2020, the idea of a powerful, destructive, and advanced species undone by an initially mundane disease seems less  of a deus ex machina every day.

## The Three Pillars of a Golden Age

Bertolt Brecht famously said that "Art is not a mirror held up to society but a hammer with which to shape it."

What do I mean when I ask "What is the Golden Age of apocalyptic literature?" Beyond the ulterior motives I have when asking it - which we will surely get back to - what I'm actually asking is two fold. When was the best time to be writing and publishing apocalyptic fiction? And when was the best time to be reading and engaging with apocalyptic fiction?

While art being a hammer and not a mirror is a vivid metaphor, I think its inarguable that a book is nothing more than the ongoing conclusion of an author's experiences both personal and societal and their memories. What could be more than that? Walter M. Miller, Jr. wrote about monastery that survived a nuclear armageddon because of his experiences bombing the Monte Cassino Monastery in a war whose end would see us opening the door to that potential future. _A Canticle for Leibowitz_ hasn't been out of print since 1959.

This is because readers are also products of their environment. The hammerees, if you will. The readers of _A Canticle for Leibowitz_ had left that same war through that same door. Maybe they liked it because the book depicted a society that had survived their worst fears. Maybe they liked it because it poked fun at the futility and narcissism of those fears. But they liked it. And, importantly, they liked it from 1959 all the way through to 2020. Something rang true for readers for 60 years running.

The conclusion I hope this leads you to will be one of the axioms I proceed with: people tend to respond positively to books they emotionally connect with. If this is true, then when we look at apocalyptic books throughout modern time we would expect a few things. When society is more afraid or anxious we would hope to see that more apocalyptic books are published; older, relevant apocalyptic books are more in demand; apocalyptic books are more favorably reviewed; and for books published during these times to continue to have success.

These fearful times - while being a bull in the china shop-like human psyche - are "Golden Ages" for apocalyptic literature. So to identify them, we will be looking for three to-be-defined criteria. The Three Pillars of a Golden Age: Quantity, Quality, and Popularity.

### Quantity

Don't let the intractability of the question of quantity deter you. We will still find interesting data points ahead.

In [323]:
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(3);
    
    let books_per_year = als::viz::get_books_per_year_percent_total("apocalyptic")?;
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Percent of Total Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2021, 0f64..0.015f64)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_labels(10)
        .y_labels(5)
        .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as f64))
        .x_desc("Year")
        .y_desc("Percent of Total Apocalyptic Books")
        .draw()?;

    let hist = Histogram::vertical(&chart)
        .style(Into::<ShapeStyle>::into(&graph_color).filled())
        .margin(1)
        .data(books_per_year);
    
    chart.draw_series(hist)?;

    Ok(())
})

Percent of Total Apocalyptic Books by Year
 
 
Percent of Total Apocalyptic Books
 
 
Year
 
 
 
0.5%
 
 
 
1%
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020

Charting the raw amount of apocalyptic books per year has two methodilogical problems. First, this completely ignores the longevity of successful apocalyptic books. Second, we don't have anything to compare this to. Presumably, the book industry is bigger now than it has ever been. So is this chart the natural result of there being more books in general? Or is there something behind this phenomenally lopsided graph? Let's tackle these problems one at a time.

The first problem can be at least addressed by tracking the number of editions for a book by year.

In [324]:
evcxr_figure((800, 800), |root| {
    let manifoldColor = Palette99::pick(4);
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Editions per Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_3d(1945..2021, 0..1500, 1945..2021)?;

        chart.with_projection(|mut p| {
            p.pitch = 0.4; 
            p.yaw = 3.95;
            p.scale = 0.7;
            p.into_matrix() // build the projection matrix
        });

        chart.configure_axes()
            .bold_grid_style(Into::<ShapeStyle>::into(&WHITE))
            .x_labels(20)
            .z_labels(20)
            .draw()?;

    let editions_by_year = als::viz::get_editions_per_year_surface("apocalyptic")?;
    chart.draw_series(
        SurfaceSeries::new(
            1945i32..2020,
            1945i32..2020,
            |x, z| *editions_by_year.get(&(*x,*z)).unwrap_or(&0i32),
            Into::<ShapeStyle>::into(&manifoldColor.mix(0.5)).filled()
        )
    )?;
    
    chart.draw_series(
        LineSeries::new(
            (1945..2020).map(|x| (x, 0, x)),
            Into::<ShapeStyle>::into(&manifoldColor)
        )
    )?;

    Ok(())
})

Editions per Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1945
 
 
 
1950
 
 
 
1955
 
 
 
1960
 
 
 
1965
 
 
 
1970
 
 
 
1975
 
 
 
1980
 
 
 
1985
 
 
 
1990
 
 
 
1995
 
 
 
2000
 
 
 
2005
 
 
 
2010
 
 
 
2015
 
 
 
2020
 
 
 
 
0
 
 
 
200
 
 
 
400
 
 
 
600
 
 
 
800
 
 
 
1000
 
 
 
1200
 
 
 
1400
 
 
 
 
1945
 
 
 
1950
 
 
 
1955
 
 
 
1960
 
 
 
1965
 
 
 
1970
 
 
 
1975
 
 
 
1980
 
 
 
1985
 
 
 
1990
 
 
 
1995
 
 
 
2000
 
 
 
2005
 
 
 
2010
 
 
 
2015
 
 
 
2020
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

There's a lot going on here and much to my annoyance, the plotting crate I'm using doesn't support labels on 3D graphs. So when I say "the X axis" I mean the years that go from left to right along the bottom right of the graph. This X axis represents the release date of the book. The line going down the middle also represents the release date. As you can see, books cannot have editions printed before their release date. "The Z axis" are the years that go from right to left along the bottom left side of the graph. This axis represents the year an edition of a book was released. Through process of elimination, "the Y axis" must then be the vertical dimension of the graph. This represents how many editions were printed of a book that year.

We can see spikes starting in 1985, and ebbing and flowing through to 2020 where not only do older books see an uptick, but editions printed of books published in 2010 and on fundamentally dominate this graph.

With all that out of the way, we can once again take a moment to reflect on how much of a recency bias there is.

Wow. It sure is a lot.

But which book or books are causing that spike? It appears to start after 2005.

In [325]:
{
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| {
        return book.year_published > 2005;
    });
    books.sort_by(|book_one, book_two| {
       return book_two.editions.len().cmp(&book_one.editions.len()) ;
    });
    
    als::viz::print_books_with_editions(books[0..5].to_vec());
}

Title,Author,Year Published,Number of Editions
"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008,322
"Mockingjay (The Hunger Games, #3)",Suzanne Collins,2010,271
"Inferno (Robert Langdon, #4)",Dan Brown,2013,271
"Catching Fire (The Hunger Games, #2)",Suzanne Collins,2009,253
"Divergent (Divergent, #1)",Veronica Roth,2011,213


We will finally pull back the curtain and show one of our front runners for most popular apocalyptic books: _The Hunger Games_. While it isn't the sole cause of the spike, it is certainly the number one contributing factor. Of the top 5 books above, 3 of them are in the same series, and one is very inspired by it. We will discuss popularity more in a later section. For now, I want to use our graph above to pretty solidly conclude that apocalyptic books appear to grow prevalent as time goes on.

Worth noting however is that _The Hunger Games_ is not solely repsonsible. We see editions skyrocketing into the thousands. This is the result of that accursed recency bias again. The sheer amount of books published during this time, regardless of if they had 1 or 100 editions, result in the spike we see.

This is all a lot of data and pictures, sure. But what I'd like to do is normalize this data and start finding some candidate Golden Ages. This will help combat the second methodilogical problem with trying to assess the quantity of apocalyptic books. While we can't know the raw percentage of the market these books cornered, Goodreads does have one helpful piece of data here: The Most Popular 200 Books per Year. Rest assured, we will revisit this graph in the Popularity pillar, but let's sneak a peek now.

In [326]:
evcxr_figure((800, 400), |root| {
    let top_200 = als::viz::get_number_of_top_200("apocalyptic")?;
    let bottom_190 = als::viz::get_number_of_bottom_190("apocalyptic")?;
    
    let color_one = Palette99::pick(7);
    let color_two = Palette99::pick(5);

    let mut chart = ChartBuilder::on(&root)
        .caption("Number of Most Popular 200 Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2021, 0..25)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_desc("Year")
        .y_desc("Number of Most Popular Books")
        .draw()?;

    let hist_one = Histogram::vertical(&chart)
        .style(color_one.filled())
        .margin(1)
        .data(top_200);
    
    let hist_two = Histogram::vertical(&chart)
        .style(color_two.filled())
        .margin(1)
        .data(bottom_190);
    
    chart.draw_series(hist_one)?
        .label("Top 10")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_one).filled()));
    chart.draw_series(hist_two)?
        .label("Top 200")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_two).filled()));
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    
    Ok(())
})

Number of Most Popular 200 Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Number of Most Popular Books
 
 
Year
 
 
 
 
 
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Top 10
 
 
Top 200

This graph may be more interesting than any other to come, including itself when it comes back up again later. Its supremacy comes in the form of making my job going forward a lot easier.

First, the chart itself. The Goodreads Most Popular 200 lists aren't perfect. Especially the older the older ones. I think very few people would pick a physical fight with you if you were to say, "Either _Animal Farm_ or _Pippi Longstocking_ is the most emblematic book of 1945." But if both grade school bullies and anthropomorphized 2020 have taught me anything, its not to ever assume anyone needs a reason to fight you (or throw rocks at you). But on the bottom of the Most Popular Books of 1945 List appears to be [a Smith and Wesson Reference Book](https://www.goodreads.com/book/show/16277559-smith-wesson-hand-guns), which depending on your outsider political views may or may not be the most representative book of 1945.

But with that out of the way, I can show you what makes this graph so useful.

In [327]:
evcxr_figure((800, 400), |root| {
    let top_200 = als::viz::get_number_of_top_200("apocalyptic")?;
    let bottom_190 = als::viz::get_number_of_bottom_190("apocalyptic")?;
    
    let color_one = Palette99::pick(7);
    let color_two = Palette99::pick(5);

    let mut chart = ChartBuilder::on(&root)
        .caption("Number of Most Popular 200 Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2021, 0..25)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_desc("Year")
        .y_desc("Number of Most Popular Books")
        .draw()?;

    let hist_one = Histogram::vertical(&chart)
        .style(color_one.filled())
        .margin(1)
        .data(top_200);
    
    let hist_two = Histogram::vertical(&chart)
        .style(color_two.filled())
        .margin(1)
        .data(bottom_190);
    
    chart.draw_series(hist_one)?
        .label("Top 10")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_one).filled()));
    chart.draw_series(hist_two)?
        .label("Top 200")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_two).filled()));
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    
    //1949-1955
    root.draw(
        &Rectangle::new(
            [(95, 300), (155, 360)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    
    //1962-1968
    root.draw(
        &Rectangle::new(
            [(220, 300), (288, 360)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    
    //1982-1987
    root.draw(
        &Rectangle::new(
            [(408, 230), (468, 360)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    
    //2004-2009
    root.draw(
        &Rectangle::new(
            [(618, 110), (675, 360)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    
    //2011-2014
    root.draw(
        &Rectangle::new(
            [(685, 35), (724, 360)],
            Into::<ShapeStyle>::into(&BLACK)
        )
    )?;
    
    Ok(())
})

Number of Most Popular 200 Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Number of Most Popular Books
 
 
Year
 
 
 
 
 
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Top 10
 
 
Top 200

Behold! My arbitrary Golden Age candidates! With absolutely no statistical basis, I looked at the original graph and tried to identify distinct 4-6 year periods which showed some sort of consistency or outliers. Why 4-6 years? Because I get to make up the rules. Why 5 ages? 5 seemed like a nice number, and means we only will have to eliminate 4 of them in the following sections to get our answer. And as a lazy person, this seems great.

If I were to guess at what this data looks like a little more normalized, I wouldn't surprised to see that apocalyptic fiction has a cyclic popularity. But maybe that's only because popularity and media is inherently cyclic. But before we move on with these candidates, I want to take a brief moment to discuss these nadirs.

1969-1981 saw very little apocalyptic fiction. Maybe the growing political activity of the 60s saw more people trying to process incremental changes to specific problems, such as ending the Vietnam War, and less so world ending problems. Maybe a post-Cuban Missile Crisis world had moved past it's nuclear anxiety. Maybe Tom Wolfe was right in calling the 1970s "The 'Me' Decade." Or just maybe, it's impossible to summarize or psychoanalyze a decade in any amount of words let alone a paragraph.

One of the most interesting things to me is that one of the most apocalyptic books, - and we'll be getting back to how we quantify "apocalypticness" later - a book that I thought for sure would be dominating this conversation, came out during this time. _The Stand_ by Stephen King from 1978. Though I can't say I would have particularly enjoyed focusing on a book about the end of the world being brought on by a flu like disease given recent circumstances...

1988-2003 is, without a doubt, the most interesting lull for me. Here is a sustained 15 year period where - without the ability to see the rest of the graph ahead of them - some might believe apocalyptica to be past it's prime. Here is also the age that I was born and raised in. I was born the same year _Children of Men_ by P.D. James was released which, despite the premise of the book, was a perfectly good year to be born.

The media I remember from my childhood that talked about the end of the world came far more often in the form of movies, television, and games. This gives the impression that I just wasn't much of a reader as a child, but nothing could be further from the truth. Unfortunately, anecdotal evidence is not real evidence. Fortunately, I don't care.

My traumatic brain injuries wouldn't happen until well into my teens and 20s, so I remember my childhood well. I remember reading almost every book in my school library, skipping only the abridged versions of books; except the Jules Verne illustrated and abridged collection, which actually rules pretty hard. The only contemporary books about the end of the world I read during all of that was _The Giver_ and _His Dark Materials_ (if I'm being generous with what counts as "apocalyptic"). I certainly read other apocalyptica, but it all came from before my time. I was a nerdy kid who loved Isaac Asimov and Ursula K. LeGuin. I still am a nerdy kid who loves Isaac Asimov and Ursula K. LeGuin. (The day she died I took off work and wept in my car.)

Weirdly none of my teachers ever stopped me from reading books that were way too mature for me. I vividly and specifically remember the gore and anxiety of _The Red Badge of Courage_ being a bit too much for young me.

My grandmother used to take me to the Corinth Johnson County Library all the time. They had a pretty decent kids reading area which also had movies you could rent. Until this section was decimated in my excitable child mind by Blockbuster's unheard of amount of choice, my grandmother would let me take whatever I wanted out of there. Including a copy of _Warriors of the Wind_, a heavily edited English version of _Nausicaä of the Valley of the Wind_, a movie featuring one of the most visually arresting - though brief - apocalypses. This may be the apocalypse most deeply rooted in my subconscious. Who can really say? I was young enough when I first watched it that I only have fleeting memories that intertwine and become confused with repeated watchings of the unedited original. But when Miramax would release Princess Mononoke for Western audiences in 2001 in theaters, how could my grandmother say no. "It was cartoon! And it's by the same people who made _Warriors of the Wind_, which he loves. Besides, what could be scary for an 8 year old in a cartoon?" I assume my grandmother asked my parents, in those exact words, no less. A lot can be scary to an 8 year old in a dark movie theater, it turns out.

Quantity may be impractical to, well, quantify. But with our candidates in hand we'll proceed anyway.

### Quality

I really didn't want to have four pillars, and luckily the English language is here to save me. I will be looking at two aspects of "quality" in this pillar: the perceived "excellence" of a book as measured by user ratings; and the "character" of a book, or "how apocalyptic" it is, as measured by how many users tagged it "apocalyptic."

You and I aren't dummies though. We're smarties. We can't just take the raw rating of a book, right?

In [328]:
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(10);
    let circle_color = Palette99::pick(6);
    let overall_color = Palette99::pick(14);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    books.sort_by(|book_one, book_two| book_two.rating.partial_cmp(&book_one.rating).unwrap());
    
    let rating_per_year = als::viz::get_shelf_rating_per_year(&books)?;
    
    let book_ratings_points = books
        .iter()
        .map(|book| {
            return Circle::new(
                (book.year_published, book.rating as f64),
                3,
                circle_color.filled()
            );
        });
    
    let average_rating_line = LineSeries::new(
        (1945i32..2021)
            .filter(|year| rating_per_year.contains_key(&year))
            .map(|year| {
                let mut rating = *rating_per_year.get(&year).unwrap_or(&0.0f32) as f64;
                return (year, rating);
            }),
            Into::<ShapeStyle>::into(&graph_color)
    );
    
    let overall_average_rating = books.iter().fold(0.0, |acc, book| acc + book.rating) / books.len() as f32;
    
    let overall_average = LineSeries::new(
        (1945..2021).map(|year| (year, overall_average_rating as f64)),
        Into::<ShapeStyle>::into(&overall_color).stroke_width(2)
    );
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Rating of Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2020, 2.8f64..5.0)?;

    chart.configure_mesh()
        .x_labels(10)
        .y_labels(5)
        .x_desc("Year")
        .y_desc("Rating")
        .draw()?;
    
    chart.draw_series(overall_average)?
        .label("Overall Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&overall_color).filled()
            );
        });

    chart.draw_series(book_ratings_points)?
        .label("Book")
        .legend(|(x, y)| Rectangle::new(
                [(x, y - 5), (x + 10, y + 5)],
                Into::<ShapeStyle>::into(&circle_color).filled()
            ));
    
    chart.draw_series(average_rating_line)?
        .label("Annual Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&graph_color).filled()
            );
        });
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    Ok(())
})

Average Rating of Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Rating
 
 
Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
<polyline fill="none" opacity="1" stroke="#800000" stroke-width="2" points="60,188 69,188 79,188 88,188 98,188 108,188 117,188 127,188 136,188 146,188 156,188 165,188 175,188 184,188 194,188 204,188 213,188 223,188 232,188 242,188 252,188 261,188 271,188 280,188 290,188 300,188 309,188 319,188 328,188 338,188 348,188 357,188 367,188 376,188 386,188 396,188 405,188 415,188 424,188 434,188 444,188 453,188 463,188 472,188 482,188 492,188 501,188 511,188 520,188 530,188 540,188 549,188 559,188 568,188 578,188 588,188 597,188 607,188 616,188 626,188 636,188 645,188 655,188 664,188 674,188 684,188 693,188 703,188 712,188 722,188 732,188 741,188 751,188 760,188 770,188 780,188 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#008080" stroke-width="1" points="60,184 79,176 98,162 117,173 127,161 136,183 146,189 156,207 165,184 175,185 184,202 194,177 213,257 223,246 232,163 242,235 252,217 261,226 271,195 280,197 290,192 309,160 319,158 338,228 348,214 357,197 367,188 376,161 386,166 396,180 405,251 415,199 424,206 434,181 444,198 453,208 463,158 472,198 482,160 492,173 501,141 511,198 

This doesn't actually look that bad, right?

Here are all of our apocalyptic books' rating. The average is also shown with the line going through. All but three of our books are rated above a 3.0, and if you don't mind the detour - presumably if you've made it this far, you are at least willing to put up with detours - I'd like to look at the top five worst rated books.

In [329]:
{    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 0);
    books.sort_by(|book_one, book_two| book_one.rating.partial_cmp(&book_two.rating).unwrap());
    
    let top_5: Vec<als::apis::goodreads::Book> = books[0..5].iter().cloned().collect();
    
    als::viz::print_books_with_ratings_and_num_ratings(top_5);
}

Title,Author,Year Published,Rating,Number of Ratings
The Flame Alphabet,Ben Marcus,2012,2.88,5014
Tumble & Fall,Alexandra Coutts,2013,2.96,1167
2012: The War For Souls,Whitley Strieber,2007,2.97,1441
"After London: or, Wild England",Richard Jefferies,1885,3.08,625
"Eruption (Supervolcano, #1)",Harry Turtledove,2011,3.12,1423


Here are a selection of my favorite parts of reviews for each book:

**The Flame Alphabet by Ben Marcus**
> Nicole, 1 star review
> > 70 pages in and this is the most boring "thriller" I've ever read. The premise is amazing, language that kills!, and apparently it's a true story because reading this is a slow painful death.

**Tumble & Fall by Alexandra Coutts**
> B, 1 star review
> > DNF--NO STARS. I stopped reading when the father bought his son a prostitute.
> 
> > Shelves: almost-fell-asleep, meh, arc, awful-ending, caught-in-a-bad-romance, judge-a-book-by-the-cover, unmet-potential, i-totally-saw-that-coming, netgalley-edelweiss-arc, was-warned-not-to-read-this

**2012: The War For Souls by Whitley Strieber**
> Steve, 1 star review
> > First off, this book sucked big time. \[...\]

> Chris Morton, 1 star review
> > The Mayan calender ends ... the war for souls begins ... load of bollocks.

**After London: or, Wild England by Richard Jefferies**
> Roo Macleod, 2 star review
> > Heavy
> > 
> > I could not get into this tomb at all. It was like reading my own obituary and finding out I was seriously boring

**Eruption by Harry Turtledove**
> karen, 1 star review
> > i really am not looking forward to writing this review. this was the worst book i have ever read, and even though david was kind enough to point out that most of the books i read are shitty, he is wrong. this one is the shittiest.

---

I thought, or maybe hoped, these would be more interesting. After London is good though; don't listen to Roo Macleod. I don't know why you would, but you're reading this so apparently you'll read and listen to anything.

Sadly though, these worst 3 don't seem all that wacky given what they could have been. Unlike our top 15 best rated books, which will show the major issue with only going by raw rating.

In [330]:
{    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 0);
    books.sort_by(|book_one, book_two| book_two.rating.partial_cmp(&book_one.rating).unwrap());
    
    let top_15: Vec<als::apis::goodreads::Book> = books[0..15].iter().cloned().collect();
    
    als::viz::print_books_with_ratings_and_num_ratings(top_15);
}

Title,Author,Year Published,Rating,Number of Ratings
"Words of Radiance (The Stormlight Archive, #2)",Brandon Sanderson,2014,4.75,203053
"The Walking Dead, Compendium 3",Robert Kirkman,2015,4.61,6915
"All the Stars in the Sky (Until the End of the World, #3)",Sarah Lyons Fleming,2015,4.58,2988
"Peripeteia (The City, #2)",Sarah Lyons Fleming,2017,4.57,1286
"Magic Triumphs (Kate Daniels, #10)",Ilona Andrews,2018,4.57,21335
"Magic Binds (Kate Daniels, #9)",Ilona Andrews,2016,4.55,27057
"The Walking Dead, Compendium 2",Robert Kirkman,2012,4.52,17015
"Magic Breaks (Kate Daniels, #7)",Ilona Andrews,2014,4.51,40104
The Hunted (The Enemy #6),Charlie Higson,2014,4.51,2840
"The Hero of Ages (Mistborn, #3)",Brandon Sanderson,2008,4.49,260944


Being the smarty you are, you probably saw this coming, but these are pretty low numbers of ratings on some of these. Along with showing some of our worst data, these ratings can't be trusted. (Sorry, Brandon Sanderson, but _Nausicaä of the Valley of the Wind_ is wildly better than _Words of Radiance_ or _The Hero of Ages_.) (Told you that _Princess Mononoke_ aside would pay off.) Though I'm honestly surprised at how many people rated - and therefore theoretically bought - _The Hunger Games Trilogy Boxset_.

So let's try this all again with a weighted rating. While doing this I found another example of how skewed this data set is. The average number of ratings a book on our list receives? 63,362. The median number of ratings? 6060! Ten times fewer. Meanwhile, the average rating is 3.92, which might make the Goodreads reviewers the kindest anywhere on the internet. I wonder if the amount of time you have to spend with a book introduces a Stockholm Syndrome-esque bias? IMDB has an average rating of 6.9 and Metacritic has an average rating between 64-74.

In [331]:
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(10);
    let circle_color = Palette99::pick(6);
    let overall_color = Palette99::pick(14);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    books.sort_by(|book_one, book_two| {
        als::viz::get_weighted_rating_for_book(
            &book_two
        ).partial_cmp(
            &als::viz::get_weighted_rating_for_book(
                &book_one
            )
        ).unwrap()
    });
    
    let rating_per_year = als::viz::get_shelf_weighted_rating_per_year(&books)?;
    
    let book_ratings_points = books
        .iter()
        .map(|book| {
            let weighted_rating = als::viz::get_weighted_rating_for_book(&book) as f64;
            return Circle::new(
                (book.year_published, weighted_rating),
                3,
                circle_color.filled()
            );
        });
    
    let average_rating_line = LineSeries::new(
        (1945i32..2021)
            .filter(|year| rating_per_year.contains_key(&year))
            .map(|year| {
                let mut rating = *rating_per_year.get(&year).unwrap_or(&0.0f32) as f64;
                return (year, rating);
            }),
            Into::<ShapeStyle>::into(&graph_color)
    );
    
    let overall_average_rating = books
        .iter()
        .fold(0.0, |acc, book| acc + als::viz::get_weighted_rating_for_book(&book) / books.len() as f32);
    
    let overall_average = LineSeries::new(
        (1945..2021).map(|year| (year, overall_average_rating as f64)),
        Into::<ShapeStyle>::into(&overall_color).stroke_width(2)
    );
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Weighted Rating of Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2020, 3.0f64..5.0)?;

    chart.configure_mesh()
        .x_labels(10)
        .y_labels(5)
        .x_desc("Year")
        .y_desc("Weighted Rating")
        .draw()?;
    
    chart.draw_series(overall_average)?
        .label("Overall Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&overall_color).filled()
            );
        });
    
    chart.draw_series(book_ratings_points)?
        .label("Book")
        .legend(|(x, y)| Rectangle::new(
                [(x, y - 5), (x + 10, y + 5)],
                Into::<ShapeStyle>::into(&circle_color).filled()
            ));
    
    chart.draw_series(average_rating_line)?
        .label("Annual Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&graph_color).filled()
            );
        });
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    Ok(())
})

Average Weighted Rating of Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Weighted Rating
 
 
Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
<polyline fill="none" opacity="1" stroke="#800000" stroke-width="2" points="60,200 69,200 79,200 88,200 98,200 108,200 117,200 127,200 136,200 146,200 156,200 165,200 175,200 184,200 194,200 204,200 213,200 223,200 232,200 242,200 252,200 261,200 271,200 280,200 290,200 300,200 309,200 319,200 328,200 338,200 348,200 357,200 367,200 376,200 386,200 396,200 405,200 415,200 424,200 434,200 444,200 453,200 463,200 472,200 482,200 492,200 501,200 511,200 520,200 530,200 540,200 549,200 559,200 568,200 578,200 588,200 597,200 607,200 616,200 626,200 636,200 645,200 655,200 664,200 674,200 684,200 693,200 703,200 712,200 722,200 732,200 741,200 751,200 760,200 770,200 780,200 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#008080" stroke-width="1" points="60,200 79,192 98,177 117,194 127,184 136,195 146,206 156,209 165,198 175,202 184,209 194,194 213,207 223,222 232,181 242,224 252,214 261,222 271,206 280,203 290,208 309,176 319,185 338,235 348,213 357,211 367,200 376,173 386,185 396,201 405,208 415,205 424,207 434,197 444,202 453,208 463,175 472,202 482,178 492,187 501,155 511,214 520,

Ah, that's better.

There's not too much different here from our earlier chart. Both resemble the heart beat of an incredibly sick person, but this one smooths out the edges and makes it more consistent. I have no idea if that would make a human being healthier or sicker, but it certainly lets us find the anomalies a lot quicker.

Speaking of anomalies, we have only only one book get above a 4.5. Any guesses?

In [332]:
{
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    
    books.sort_by(|book_one, book_two| {
        als::viz::get_weighted_rating_for_book(
            &book_two
        ).partial_cmp(
            &als::viz::get_weighted_rating_for_book(
                &book_one
            )
        ).unwrap()
    });
        
    let top_5: Vec<als::apis::goodreads::Book> = books[0..5].iter().cloned().collect();
        
    als::viz::print_books_with_weighted_ratings(top_5);
}

Title,Author,Year Published,Weighted Rating
"Words of Radiance (The Stormlight Archive, #2)",Brandon Sanderson,2014,4.7260504
"The Hero of Ages (Mistborn, #3)",Brandon Sanderson,2008,4.477144
"The Hunger Games Trilogy Boxset (The Hunger Games, #1-3)",Suzanne Collins,2008,4.4625497
"Magic Binds (Kate Daniels, #9)",Ilona Andrews,2016,4.4353695
"Winter (The Lunar Chronicles, #4)",Marissa Meyer,2015,4.43373


No points for guessing "the same book as before." What a lame guess, no matter how right it is.

If you're looking for authentic surprise from me, you've found it. I genuinely did not expect the two books I made fun of earlier to appear here now. Egg on my face, huh? I'd apologize, but I know that between my lack of anything bordering on notoriety and his frightening prolificity, there is a literal 0% chance of Mr. Sanderson ever reading this. Also, that better be one impressive box set. 187,000 people rated it as if it was.

[Oh, wow. It's extremely disappointing.](https://www.goodreads.com/book/photo/7938275-the-hunger-games-trilogy-boxset) My bar was so incredibly low that I didn't think it would hurt me like this. So the published date there is going off of the first book, which is perfectly fine. It may even be preferred data wise. But the actual box set is from 2010. I mean, it wouldn't be a movie until 2012 (even though it was optioned in 2009), but it was already a bestseller by then, Scholastic!

To step out of the bit for a second and clear this up for "scientific" purposes: I know that most of the reviews and ratings there are for the trilogy as a whole. It's important to remember that The Hunger Games changed the very nature of our questions. If I was actually here to answer questions, I could answer one of my original questions ("What was the best time to be an author of apocalyptic fiction?") with this single data point. The answer would be, "At the exact same time Suzanne Collins was writing The Hunger Games, followed closely by any time after."

Why don't we take a look at the other end of the spectrum, just for fun?

In [333]:
{
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    
    books.sort_by(|book_one, book_two| {
        als::viz::get_weighted_rating_for_book(
            &book_two
        ).partial_cmp(
            &als::viz::get_weighted_rating_for_book(
                &book_one
            )
        ).unwrap()
    });
        
    let bottom_5: Vec<als::apis::goodreads::Book> = books.iter().rev().take(5).cloned().collect();
        
    als::viz::print_books_with_weighted_ratings(bottom_5);
}

Title,Author,Year Published,Weighted Rating
"Pride and Prejudice and Zombies (Pride and Prejudice and Zombies, #1)",Seth Grahame-Smith,2009,3.3281054
"The Prophet of Yonwood (Book of Ember, #3)",Jeanne DuPrau,2006,3.386344
The Cabin at the End of the World,Paul Tremblay,2018,3.3942857
California,Edan Lepucki,2014,3.3994408
Zone One,Colson Whitehead,2011,3.4238622


Having only read _Pride and Prejudice and Zombies_ from this list, I think that may be a bit unfair. It's certainly not good, and in fact I wouldn't stop you from saying it's bad. But the worst piece of apocalyptic fiction ever? A bit harsh. Especially for a book title that uses polysyndeton in its title, which is easily the third best, greek rooted figure of speech.

The symmetry here can't be ignored though. While _Pride and Prejudice and Zombies_ may be the nadir of it's movement, as much as or even more than _The Hunger Games_ and it's wave of young adult distopian novels, Zombie Fiction will come to shape, swarm, and dominate the 21st Century.

Now that I've really paraded these graphs and tables around, they've earned a break. What about our candidate Golden Ages?

In [334]:
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(16);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    
    let mut candidates = vec![
        "1945-2020",
        "1949-1955",
        "1962-1968",
        "1982-1987",
        "2004-2009",
        "2011-2014"
    ];
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Weighted Rating for Golden Age Candidates", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(0..6, 3.8f64..4.1)?;

    chart.configure_mesh()
        .disable_mesh()
        .disable_axes()
        .label_style(TextStyle::from(("sans-serif", 20).into_font()).color(&WHITE))
        .draw()?;

    let mut data: Vec<(i32, f64)> = Vec::new();
    let mut text_data: Vec<(i32, f64)> = Vec::new();
    
    for (idx, year_string) in candidates.iter_mut().enumerate() {
        let mut year_split = year_string.split("-");
        let start_year = year_split.next().unwrap().parse::<i32>().unwrap();
        let end_year = year_split.next().unwrap().parse::<i32>().unwrap();
        
        let weighted_average = als::viz::get_shelf_weighted_rating_for_years(&books, start_year, end_year)?;
        data.push((idx as i32, weighted_average as f64));
        text_data.push((idx as i32, weighted_average as f64));
    }
    
    let hist = Histogram::vertical(&chart)
        .style(Into::<ShapeStyle>::into(&graph_color).filled())
        .margin(1)
        .data(data);
    
    chart.draw_series(hist)?;
    
    // 1945-2020
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[0].1),
            (100, 169),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[0]),
            (83, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1949-1955
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[1].1),
            (220, 138),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[1]),
            (200, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1962-1968
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[2].1),
            (340, 245),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[2]),
            (323, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1982-1987
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[3].1),
            (463, 170),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[3]),
            (445, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 2004-2009
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[4].1),
            (580, 150),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[4]),
            (567, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 2011-2014
    root.draw(
        &Text::new(
            format!("{:.3}", text_data[5].1),
            (702, 193),
            ("Arial", 18).into_font()
        )
    )?;
    root.draw(
        &Text::new(
            format!("{}", candidates[5]),
            (687, 370),
            ("Arial", 18).into_font()
        )
    )?;

    Ok(())
})

Average Weighted Rating for Golden Age Candidates
 
 
3.8
 
 
3.85
 
 
3.9
 
 
3.95
 
 
4.0
 
 
4.05
 
 
0
 
 
1
 
 
2
 
 
3
 
 
4
 
 
5
 
 
6
 
 
 
 
 
 
 
 
3.956
 
 
1945-2020
 
 
3.984
 
 
1949-1955
 
 
3.887
 
 
1962-1968
 
 
3.956
 
 
1982-1987
 
 
3.973
 
 
2004-2009
 
 
3.934
 
 
2011-2014

Less than .100 separated our highest and lowest score, so this really is neck and neck.

The first bar on this chart represents our overall average weighted rating. Only two of our candidates manage to average out higher: 1949-1955 and 2004-2009.

_1984_ by George Orwell is the highest rated book of 1949-1955, and _The Hero of Ages_ by Brandon Sanderson is the highest rated book of 2004-2009. I think it's a bit unfair to compare these two. _The Hero of Ages_ is fantasy novel about a girl who gains enough magical power to be able to literally destroy and rebuild the world. While certainly apocalyptic in imagery and metaphor, a book so separted from reality doesn't pack quite the same punch in 2020 as a book about a post-truth, totalitarian dictatorship does.

If you think comparing those two is unfair though, the second highest rated book of 2004-2009 will provide you no respite. _The Hunger Games Trilogy_ by Suzanne Collins is more or less _1984_ meets the film _Battle Royale_. This comparison I think demonstrates what makes crowning an earlier candidate "The Golden Age of Apocalyptic Fiction" so tempting. The societal fears being uncovered by authors and devoured by readers at the beginning of the modern era were so primordial in nature that almost a century later they remain relatively unchanged and undiluted.

However, this origin seeking is an endless goalpost-moving cycle. _The Hunger Games_ couldn't have existed without _1984_, which couldn't have existed without _A Brave New World_, which couldn't have existed without _We_ by Yevgeny Zamyatin and a whole lot of drugs. (Actually, despite being a huge (the hugest?) proponent of mescaline, Huxley didn't start taking psychedelics until 1953.)

Unlike Survivor of The Bachelor, and contrary to what I said earlier, no contestants will be getting cut here. Which is good for you, 1962-1968. Get it together, buddy. Do it for _Cat's Cradle_.

Solely for archival purposes, here are the Top 5 highest rated books of each candidate golden age.

In [335]:
{
    println!("EVCXR_BEGIN_CONTENT text/html\n");
    let candidates = [
        (1949, 1955),
        (1962, 1968),
        (1982, 1987),
        (2004, 2009),
        (2011, 2014)
    ];

    for candidate in candidates.iter() {
        println!("<h3>{}-{}</h2>", candidate.0, candidate.1);
        println!("<table style=\"width:100%\"><tbody>");
        println!("<tr><td><b>Title</b></td><td><b>Author</b></td><td><b>Year Published</b></td><td><b>Weighted Rating</b></td></tr>");
        let mut books = als::viz::get_books("apocalyptic")?;
        books.retain(|book| book.year_published >= candidate.0 && book.year_published <= candidate.1);
    
        books.sort_by(|book_one, book_two| {
            als::viz::get_weighted_rating_for_book(
                &book_two
            ).partial_cmp(
                &als::viz::get_weighted_rating_for_book(
                    &book_one
                )
            ).unwrap()
        });

        let top_5: Vec<als::apis::goodreads::Book> = books[0..5].iter().cloned().collect();

        for book in top_5.iter() {
            println!(
                "{}",
                format!(
                    "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>",
                    book.title,
                    book.author,
                    book.year_published,
                    als::viz::get_weighted_rating_for_book(&book)
                )
            );
        }
        println!("</tbody></table>");
    }
    println!("\nEVCXR_END_CONTENT");
}

Title,Author,Year Published,Weighted Rating
1984,George Orwell,1949,4.179501
The Martian Chronicles,Ray Bradbury,1949,4.133895
Childhood's End,Arthur C. Clarke,1953,4.10163
I Am Legend,Richard Matheson,1954,4.0611696
City,Clifford D. Simak,1952,4.045445
Title,Author,Year Published,Weighted Rating
Cat's Cradle,Kurt Vonnegut Jr.,1963,4.155829
Do Androids Dream of Electric Sheep?,Philip K. Dick,1968,4.077279
A Clockwork Orange,Anthony Burgess,1962,3.9893327
The Wall,Marlen Haushofer,1963,3.974742


If we look at the data Goodreads gives us, we can interpret one data point in an especially interesting way: how many times a book is shelved on a specific shelf. If we can accept that however many times Goodreads users tag a book as "apocalyptic" represents a rough estimate of that book's apocalytpic-ness, then we can begin to inspect the apocalyptic quality of books throughout time.

We've learned our lesson from ratings though. The sheer number of times someone tagged a book apocalyptic isn't really a fair measure. Ideally, we would look at the all of the tags a book has and see what percentage of those are "apocalyptic." Goodreads doesn't really provide a good way of doing that though, and I'm very lazy. Instead, I'll be proceeding with the assumption that rating a book is more or less the most basic and atomic interaction you can have with a book. So if we look at apocalypticness as the ratio of how many people tagged a book apocalyptic to how many people rated it, we should end up with a similar measure.

I am also proceeding with the assumption that the number of people who tag a book with a specific genre before finishing it is around the same percentage that rate it before finishing it.

In [336]:
// shelved number / num ratings by year
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(18);
    let circle_color = Palette99::pick(19);
    let overall_color = Palette99::pick(21);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    //books.sort_by(|book_one, book_two| book_two.rating.partial_cmp(&book_one.rating).unwrap());
    
    let shelf_percentage_per_year = als::viz::get_shelf_shelved_percentage_per_year(&books)?;
    
    let book_shelf_percentage_points = books
        .iter()
        .map(|book| {
            return Circle::new(
                (book.year_published, als::viz::get_book_shelf_percentage(&book) as f64),
                3,
                circle_color.filled()
            );
        });
    
    let average_shelf_percentage_line = LineSeries::new(
        (1945i32..2021)
            .filter(|year| shelf_percentage_per_year.contains_key(&year))
            .map(|year| {
                let mut shelf_percentage = *shelf_percentage_per_year.get(&year).unwrap_or(&0.0f32) as f64;
                return (year, shelf_percentage);
            }),
            Into::<ShapeStyle>::into(&graph_color)
    );
    
    let overall_shelf_percentage_rating = books.iter().fold(0.0, |acc, book| acc + als::viz::get_book_shelf_percentage(&book)) / books.len() as f32;

    let overall_shelf_percentage = LineSeries::new(
        (1945..2021).map(|year| (year, overall_shelf_percentage_rating as f64)),
        Into::<ShapeStyle>::into(&overall_color).stroke_width(2)
    );
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Shelf Percentage of Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2020, 0.0f64..5.0)?;

    chart.configure_mesh()
        .x_labels(10)
        .y_labels(5)
        .x_desc("Year")
        .y_desc("Shelf Percentage")
        .draw()?;
    
    chart.draw_series(overall_shelf_percentage)?
        .label("Overall Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&overall_color).filled()
            );
        });

    chart.draw_series(book_shelf_percentage_points)?
        .label("Book")
        .legend(|(x, y)| Rectangle::new(
                [(x, y - 5), (x + 10, y + 5)],
                Into::<ShapeStyle>::into(&circle_color).filled()
            ));
    
    chart.draw_series(average_shelf_percentage_line)?
        .label("Annual Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&graph_color).filled()
            );
        });
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    Ok(())
})

Average Shelf Percentage of Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Shelf Percentage
 
 
Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
<polyline fill="none" opacity="1" stroke="#E6194B" stroke-width="2" points="60,329 69,329 79,329 88,329 98,329 108,329 117,329 127,329 136,329 146,329 156,329 165,329 175,329 184,329 194,329 204,329 213,329 223,329 232,329 242,329 252,329 261,329 271,329 280,329 290,329 300,329 309,329 319,329 328,329 338,329 348,329 357,329 367,329 376,329 386,329 396,329 405,329 415,329 424,329 434,329 444,329 453,329 463,329 472,329 482,329 492,329 501,329 511,329 520,329 530,329 540,329 549,329 559,329 568,329 578,329 588,329 597,329 607,329 616,329 626,329 636,329 645,329 655,329 664,329 674,329 684,329 693,329 703,329 712,329 722,329 732,329 741,329 751,329 760,329 770,329 780,329 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#000080" stroke-width="1" points="60,358 79,358 98,342 117,340 127,355 136,351 146,352 156,313 165,332 175,329 184,342 194,308 213,56 223,301 232,353 242,329 252,253 261,341 271,345 280,349 290,358 309,355 319,330 338,332 348,324 357,343 367,300 376,349 386,315 396,346 405,176 415,350 424,248 434,323 444,300 453,261 463,351 472,324 482

Overall, apocalyptic books are tagged as such 0.48% of the time. That means roughly 1 out of every 208 users rating the book will file it away on their "apocalyptic" shelf. This is honestly way more than I thought it would be. A lot of this is due to the fact that many books receive very few ratings. Only 4 books manage to have more than 4% of their raters tag them apocalyptic:

- _Elixir_ by Anna Abner from 2014 with 4.97%
- _Some Will Not Die_ by Algis Budrys from 1961 with 4.52%
- _L.A. Dark (Jeremy's Run)_ by G.F. Gustav from 2013 with 4.40%
- _The Last Gasp_ by Trevor Hoyle from 1983 with 4.04%

The most important fact to point out about these is bad of a title _L.A. Dark (Jeremy's Run)_ is. It's actually so bad that it's gone around the bend and I now enjoy it out of some sense of masochism. The second most important fact to point out is that these four books received less than 1000 ratings combined.

While this is bit of a raisin in the cookie of being able to derive useful learnings out of the data, I still do believe in the power of averages and regression to the mean. So let's look at two aspects of our candidates.

In [337]:
// shelved number / num ratings by candidates
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(24);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    
    let mut candidates = vec![
        "1945-2020",
        "1949-1955",
        "1962-1968",
        "1982-1987",
        "2004-2009",
        "2011-2014"
    ];
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Shelf Percentage for Golden Age Candidates", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(0..6, 0.0f64..1.0)?;

    chart.configure_mesh()
        .disable_mesh()
        .disable_axes()
        .label_style(TextStyle::from(("sans-serif", 20).into_font()).color(&WHITE))
        .draw()?;

    let mut data: Vec<(i32, f64)> = Vec::new();
    let mut text_data: Vec<(i32, f64)> = Vec::new();
    
    for (idx, year_string) in candidates.iter_mut().enumerate() {
        let mut year_split = year_string.split("-");
        let start_year = year_split.next().unwrap().parse::<i32>().unwrap();
        let end_year = year_split.next().unwrap().parse::<i32>().unwrap();
        
        let weighted_average = als::viz::get_shelf_percentage_for_years(&books, start_year, end_year)?;
        data.push((idx as i32, weighted_average as f64));
        text_data.push((idx as i32, weighted_average as f64));
    }
    
    let hist = Histogram::vertical(&chart)
        .style(Into::<ShapeStyle>::into(&graph_color).filled())
        .margin(1)
        .data(data);
    
    chart.draw_series(hist)?;
    
    // 1945-2020
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[0].1),
            (100, 179),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[0]),
            (83, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1949-1955
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[1].1),
            (220, 260),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[1]),
            (200, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1962-1968
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[2].1),
            (340, 170),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[2]),
            (323, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1982-1987
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[3].1),
            (463, 75),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[3]),
            (445, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 2004-2009
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[4].1),
            (580, 238),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[4]),
            (567, 370),
            ("Arial", 18).into_font()
        )
    )?;

    // 2011-2014
    root.draw(
        &Text::new(
            format!("{:.3}%", text_data[5].1),
            (702, 170),
            ("Arial", 18).into_font()
        )
    )?;
    root.draw(
        &Text::new(
            format!("{}", candidates[5]),
            (687, 370),
            ("Arial", 18).into_font()
        )
    )?;

    Ok(())
})

Average Shelf Percentage for Golden Age Candidates
 
 
0.1
 
 
0.2
 
 
0.3
 
 
0.4
 
 
0.5
 
 
0.6
 
 
0.7
 
 
0.8
 
 
0.9
 
 
0
 
 
1
 
 
2
 
 
3
 
 
4
 
 
5
 
 
6
 
 
 
 
 
 
 
 
0.483%
 
 
1945-2020
 
 
0.243%
 
 
1949-1955
 
 
0.508%
 
 
1962-1968
 
 
0.795%
 
 
1982-1987
 
 
0.303%
 
 
2004-2009
 
 
0.514%
 
 
2011-2014

This chart has some expected and some unexpected revelations. For the first two ages, this smaller percentage makes sense to me. We will see this born out in the data and visualized later, but, for the most part, older books have more ratings. This will inevitably lead to them having a smaller percentage of people tagging them anything let alone "apocalyptic."

Out of _1984_'s 3,109,403 ratings, only 56 people tagged it "apocalyptic." Seemingly, the more often a book is rated, the more unhelpful it's tags become. On the one hand, _1984_ is certainly more about its post-apocalypse than its actual apocalypse. On the other hand, saying _1984_ is only 0.0018% apocalyptic seems a bit of a lowball. I imagine that the rest of _1984_'s tags would be similarly diluted.

On the other hand, 1982-1987 is certainly an outlier, and I wouldn't have expected 2004-2009 to do so poorly. But when you remember that the most rated book on our list, _The Hunger Games_, is from 2008 you can see our "more book ratings equals less helpful genres" rule begin to apply itself again.

Unfortunately, deciding to excise number of ratings to overcome this rule will reveal a paradox.

In [338]:
// shelved number by candidates
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(35);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    
    let mut candidates = vec![
        "1945-2020",
        "1949-1955",
        "1962-1968",
        "1982-1987",
        "2004-2009",
        "2011-2014"
    ];
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Times Shelved for Golden Age Candidates", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(0..6, 0..65)?;

    chart.configure_mesh()
        .disable_mesh()
        .disable_axes()
        .label_style(TextStyle::from(("sans-serif", 20).into_font()).color(&WHITE))
        .draw()?;

    let mut data: Vec<(i32, i32)> = Vec::new();
    let mut text_data: Vec<(i32, i32)> = Vec::new();
    
    for (idx, year_string) in candidates.iter_mut().enumerate() {
        let mut year_split = year_string.split("-");
        let start_year = year_split.next().unwrap().parse::<i32>().unwrap();
        let end_year = year_split.next().unwrap().parse::<i32>().unwrap();
        
        let average = als::viz::get_shelf_num_for_years(&books, start_year, end_year)?;
        data.push((idx as i32, average));
        text_data.push((idx as i32, average));
    }
    
    let hist = Histogram::vertical(&chart)
        .style(Into::<ShapeStyle>::into(&graph_color).filled())
        .margin(1)
        .data(data);
    
    chart.draw_series(hist)?;
    
    // 1945-2020
    root.draw(
        &Text::new(
            format!("{}", text_data[0].1),
            (105, 155),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[0]),
            (83, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1949-1955
    root.draw(
        &Text::new(
            format!("{}", text_data[1].1),
            (225, 40),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[1]),
            (200, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1962-1968
    root.draw(
        &Text::new(
            format!("{}", text_data[2].1),
            (345, 220),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[2]),
            (323, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1982-1987
    root.draw(
        &Text::new(
            format!("{}", text_data[3].1),
            (468, 220),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[3]),
            (445, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 2004-2009
    root.draw(
        &Text::new(
            format!("{}", text_data[4].1),
            (585, 140),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[4]),
            (567, 370),
            ("Arial", 18).into_font()
        )
    )?;

    // 2011-2014
    root.draw(
        &Text::new(
            format!("{}", text_data[5].1),
            (707, 220),
            ("Arial", 18).into_font()
        )
    )?;
    root.draw(
        &Text::new(
            format!("{}", candidates[5]),
            (687, 370),
            ("Arial", 18).into_font()
        )
    )?;

    Ok(())
})

Average Times Shelved for Golden Age Candidates
 
 
0
 
 
10
 
 
20
 
 
30
 
 
40
 
 
50
 
 
60
 
 
0
 
 
1
 
 
2
 
 
3
 
 
4
 
 
5
 
 
6
 
 
 
 
 
 
 
 
36
 
 
1945-2020
 
 
58
 
 
1949-1955
 
 
23
 
 
1962-1968
 
 
23
 
 
1982-1987
 
 
39
 
 
2004-2009
 
 
23
 
 
2011-2014

Here are our candidate's books' average number of apocalyptic tags. Immediately, you'll notice that this graph is the mirror version of our previous graph. It's our shelf percentage graph, but evil and sporting a fashionable goatee.

Maybe my readers who are better at statistics saw this flip coming. But if they're _so smart_, can those same number warlocks tell me why 23 is the average for three of our candidates? Non-jokingly, it's kind of freaking me out and I'd love a rational explanation for this that's more satisfactory than "coincidence."

Alas, the world is full of coincidences. When my family took a road trip visiting potential colleges, we were in Memphis outside The Peabody Hotel. We were there for a very specific parade. Lights froms cameras flashed as an oddly dressed man stepped into the too small lobby and lead a group of ducks into the fountain. Later that same day my brother would claim he had seen Larry Byrd at that same hotel. Byrd post birds. What a coincidence.

Before we move on to Popularity, I want to acknowledge the cream of our apocalypticness crop. Despite the problems we've had with the shelf number as a data point, I do believe our top 10 most apocalyptic books, as measured by the number of times they were tagged as apocalyptic, are a pretty fair guesstimate for the top 10 most apocalyptic books.

In [339]:
{
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    
    books.sort_by(|book_one, book_two| {
        return book_two.shelf_num.cmp(&book_one.shelf_num);
    });
        
    let top_10: Vec<als::apis::goodreads::Book> = books.iter().take(10).cloned().collect();
        
    println!("EVCXR_BEGIN_CONTENT text/html\n");
    println!("<table style=\"width:100%\"><tbody>");
    println!("<tr><td><b>Title</b></td><td><b>Author</b></td><td><b>Year Published</b></td><td><b>Shelved Number</b></td></tr>");
    
    for book in top_10.iter() {
        println!(
            "{}",
            format!(
                "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>",
                book.title,
                book.author,
                book.year_published,
                book.shelf_num
            )
        );
    }
    
    println!("</tbody></table>");
    println!("\nEVCXR_END_CONTENT");
}

Title,Author,Year Published,Shelved Number
The Road,Cormac McCarthy,2006,573
Station Eleven,Emily St. John Mandel,2014,505
The Stand,Stephen King,1978,491
World War Z: An Oral History of the Zombie War,Max Brooks,2006,478
"The Passage (The Passage, #1)",Justin Cronin,2010,395
"The 5th Wave (The 5th Wave, #1)",Rick Yancey,2013,353
"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008,318
"Oryx and Crake (MaddAddam, #1)",Margaret Atwood,2003,285
"Life As We Knew It (Last Survivors, #1)",Susan Beth Pfeffer,2006,281
"Angelfall (Penryn & the End of Days, #1)",Susan Ee,2011,256


Beyond the recency bias we've now covered at length, I don't think it's a stretch to call _The Road_ by Cormac McCarthy the most apocalyptic book of all time. It's certainly the bleakest. Nor do I think it's a leap to call _The Stand_ by Stephen King the third most apocalyptic book of all time. It's even gone up in status in 2020. I have nothing against _Station Eleven_, by the way. I also have nothing in favor of it. I've never heard of it before this project.

Now that we've separated our books' shelf number from the number of ratings it received, what does the reverse look like? We'll be looking at number of ratings through the lens of...

### Popularity

Measuring the popularity of apocalyptic fiction throughout the modern era would require contemporary data that we just don't have. Blame the cowards at The New York Times who haven't made their historic best seller book list publicly available.

Measuring how popular a book was at the time of its release is our obvious best case scenario, and would probably more accurately reflect a period's golden-age-ness. But measuring a book's lasting popularity is still a valuable piece of information. This can still help us find the answers we seek (due mostly to not truly knowing the questions we're trying answer) and may reveal to us which past apocalypses still remain in the forefront of our collective unconscious.

In [340]:
// num ratings per year
// shelved number / num ratings by year
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(25);
    let circle_color = Palette99::pick(24);
    let overall_color = Palette99::pick(22);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    //books.sort_by(|book_one, book_two| book_two.rating.partial_cmp(&book_one.rating).unwrap());
    
    let num_ratings_per_year = als::viz::get_shelf_num_ratings_per_year(&books)?;
    
    let book_num_ratings_points = books
        .iter()
        .map(|book| {
            return Circle::new(
                (book.year_published, book.num_ratings as i32),
                3,
                circle_color.filled()
            );
        });
    
    let average_num_ratings_line = LineSeries::new(
        (1945i32..2021)
            .filter(|year| num_ratings_per_year.contains_key(&year))
            .map(|year| {
                let mut num_ratings = *num_ratings_per_year.get(&year).unwrap_or(&0i32);
                return (year, num_ratings);
            }),
            Into::<ShapeStyle>::into(&graph_color)
    );
    
    let overall_num_ratings_rating = books.iter().fold(0, |acc, book| acc + book.num_ratings as i32) / books.len() as i32;
    
    let overall_num_ratings = LineSeries::new(
        (1945..2021).map(|year| (year, overall_num_ratings_rating)),
        Into::<ShapeStyle>::into(&overall_color).stroke_width(2)
    );
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Number of Ratings of Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2020, 0..3000000)?;

    chart.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .y_label_formatter(&|y| format!("{:.1}m", *y as f32 / 1000000 as f32))
        .x_desc("Year")
        .y_desc("Shelf Percentage")
        .draw()?;
    
    chart.draw_series(overall_num_ratings)?
        .label("Overall Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&overall_color).filled()
            );
        });

    chart.draw_series(book_num_ratings_points)?
        .label("Book")
        .legend(|(x, y)| Rectangle::new(
                [(x, y - 5), (x + 10, y + 5)],
                Into::<ShapeStyle>::into(&circle_color).filled()
            ));
    
    chart.draw_series(average_num_ratings_line)?
        .label("Annual Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&graph_color).filled()
            );
        });
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    Ok(())
})

Average Number of Ratings of Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Shelf Percentage
 
 
Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0m
 
 
 
0.5m
 
 
 
1.0m
 
 
 
1.5m
 
 
 
2.0m
 
 
 
2.5m
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
<polyline fill="none" opacity="1" stroke="#3CB44B" stroke-width="2" points="60,351 69,351 79,351 88,351 98,351 108,351 117,351 127,351 136,351 146,351 156,351 165,351 175,351 184,351 194,351 204,351 213,351 223,351 232,351 242,351 252,351 261,351 271,351 280,351 290,351 300,351 309,351 319,351 328,351 338,351 348,351 357,351 367,351 376,351 386,351 396,351 405,351 415,351 424,351 434,351 444,351 453,351 463,351 472,351 482,351 492,351 501,351 511,351 520,351 530,351 540,351 549,351 559,351 568,351 578,351 588,351 597,351 607,351 616,351 626,351 636,351 645,351 655,351 664,351 674,351 684,351 693,351 703,351 712,351 722,351 732,351 741,351 751,351 760,351 770,351 780,351 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#F58230" stroke-width="1" points="60,56 79,340 98,234 117,353 127,357 136,292 146,267 156,356 165,346 175,355 184,358 194,354 213,358 223,345 232,339 242,358 252,358 261,358 271,357 280,345 290,334 309,353 319,357 338,357 348,358 357,356 367,356 376,324 386,315 396,354 405,358 415,34

The average apocalyptic book receives 147,293 ratings for an average rating of 3.92 out of 5.

As we noted earlier, older books tend to have more ratings. I believe this would be true with any dataset. In a better dataset, this would hopefully be because the longer a book is around, the more chance it has to accrue ratings. In our stupid Goodreads dataset though, since most of our reviews come from 2013-2016, this is due to other circumstances. Older books tend to have more editions and more availability, meaning they are more likely to be found on library shelves and are also typically cheaper. Survivorship bias also causes the older data to exist solely due to popularity; many apocalyptic books and stories from the 40s-60s were lost in the shuffle of decades, but books that were included in the newer dataset were books that were in our popular conscious enough to be remembered. I also suspect many of the older books on Goodreads with high numbers of ratings are books that were a part of our curriculum in school.

One interesting aspect of the chart is that there's no real pattern to the books that make it past one million ratings. There's 4 between 1945-1955, 4 between 1979-1993, and 4 between 2008-2011. There's a bit of a parabolic line you could conceivably draw through these, but only if you're a fan of ignoring 90% of your data.

The other line you could highlight in this data looks a lot like our total number of books histogram with the apex being shifted to 2010-2013.

While there are peaks here and there, we can see that most books are far under our average number of ratings. The average is brought up by only 28 books that have more than 500,000 ratings and only 98 that have more than the average. The only sustained period of more than two years above the average is 1945-1950.

I've pulled the wool over your eyes a bit here though. This graph is zoomed in, in order to show the relevant bits of data. You have to zoom a pretty sizeable distance away in order to see everything in its place. How far?

In [341]:
// num ratings per year
// shelved number / num ratings by year
evcxr_figure((900, 600), |root| {
    let graph_color = Palette99::pick(25);
    let circle_color = Palette99::pick(24);
    let overall_color = Palette99::pick(22);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    books.retain(|book| book.year_published > 1944);
    //books.sort_by(|book_one, book_two| book_two.rating.partial_cmp(&book_one.rating).unwrap());
    
    let num_ratings_per_year = als::viz::get_shelf_num_ratings_per_year(&books)?;
    
    let book_num_ratings_points = books
        .iter()
        .map(|book| {
            return Circle::new(
                (book.year_published, book.num_ratings as i32),
                3,
                circle_color.filled()
            );
        });
    
    let average_num_ratings_line = LineSeries::new(
        (1945i32..2021)
            .filter(|year| num_ratings_per_year.contains_key(&year))
            .map(|year| {
                let mut num_ratings = *num_ratings_per_year.get(&year).unwrap_or(&0i32);
                return (year, num_ratings);
            }),
            Into::<ShapeStyle>::into(&graph_color)
    );
    
    let overall_num_ratings_rating = books.iter().fold(0, |acc, book| acc + book.num_ratings as i32) / books.len() as i32;
    
    let overall_num_ratings = LineSeries::new(
        (1945..2021).map(|year| (year, overall_num_ratings_rating)),
        Into::<ShapeStyle>::into(&overall_color).stroke_width(2)
    );
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Number of Ratings of Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2020, 0..6500000)?;

    chart.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .y_label_formatter(&|y| format!("{}m", y / 1000000))
        .x_desc("Year")
        .y_desc("Shelf Percentage")
        .draw()?;
    
    chart.draw_series(overall_num_ratings)?
        .label("Overall Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&overall_color).filled()
            );
        });

    chart.draw_series(book_num_ratings_points)?
        .label("Book")
        .legend(|(x, y)| Rectangle::new(
                [(x, y - 5), (x + 10, y + 5)],
                Into::<ShapeStyle>::into(&circle_color).filled()
            ));
    
    chart.draw_series(average_num_ratings_line)?
        .label("Annual Average")
        .legend(|(x, y)| {
            return Rectangle::new(
                [(x, y - 6), (x + 10, y + 4)],
                Into::<ShapeStyle>::into(&graph_color).filled()
            );
        });
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    Ok(())
})

Average Number of Ratings of Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Shelf Percentage
 
 
Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0m
 
 
 
1m
 
 
 
2m
 
 
 
3m
 
 
 
4m
 
 
 
5m
 
 
 
6m
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
<polyline fill="none" opacity="1" stroke="#3CB44B" stroke-width="2" points="60,553 70,553 81,553 92,553 103,553 114,553 125,553 136,553 147,553 158,553 169,553 180,553 191,553 202,553 213,553 224,553 234,553 245,553 256,553 267,553 278,553 289,553 300,553 311,553 322,553 333,553 344,553 355,553 366,553 377,553 388,553 398,553 409,553 420,553 431,553 442,553 453,553 464,553 475,553 486,553 497,553 508,553 519,553 530,553 541,553 552,553 562,553 573,553 584,553 595,553 606,553 617,553 628,553 639,553 650,553 661,553 672,553 683,553 694,553 705,553 716,553 726,553 737,553 748,553 759,553 770,553 781,553 792,553 803,553 814,553 825,553 836,553 847,553 858,553 869,553 880,553 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#F58230" stroke-width="1" points="60,335 81,545 103,467 125,555 136,557 147,509 158,491 169,557 180,549 191,556 202,558 213,555 234,558 245,548 256,544 267,558 278,558 289,558 300,558 311,549 322,540 344,554 355,557 377,557 388,558 398,557 409,557 420,533 431,527 442,556 453,

This far.

Curse you, Jennifer Lawrence.

In [342]:
// num rating by candidates
// shelved number / num ratings by candidates
evcxr_figure((800, 400), |root| {
    let graph_color = Palette99::pick(26);
    
    let mut books = als::viz::get_books("apocalyptic")?;
    
    let mut candidates = vec![
        "1945-2020",
        "1949-1955",
        "1962-1968",
        "1982-1987",
        "2004-2009",
        "2011-2014"
    ];
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Average Number of Ratings for Golden Age Candidates", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(0..6, 0..500000)?;

    chart.configure_mesh()
        .disable_mesh()
        .disable_axes()
        .label_style(TextStyle::from(("sans-serif", 20).into_font()).color(&WHITE))
        .draw()?;

    let mut data: Vec<(i32, i32)> = Vec::new();
    let mut text_data: Vec<(i32, i32)> = Vec::new();
    
    for (idx, year_string) in candidates.iter_mut().enumerate() {
        let mut year_split = year_string.split("-");
        let start_year = year_split.next().unwrap().parse::<i32>().unwrap();
        let end_year = year_split.next().unwrap().parse::<i32>().unwrap();
        
        let average = als::viz::get_num_ratings_for_years(&books, start_year, end_year)?;
        data.push((idx as i32, average));
        text_data.push((idx as i32, average));
    }
    
    let hist = Histogram::vertical(&chart)
        .style(Into::<ShapeStyle>::into(&graph_color).filled())
        .margin(1)
        .data(data);
    
    chart.draw_series(hist)?;
    
    // 1945-2020
    root.draw(
        &Text::new(
            format!("{}", text_data[0].1),
            (95, 240),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[0]),
            (83, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1949-1955
    root.draw(
        &Text::new(
            format!("{}", text_data[1].1),
            (215, 50),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[1]),
            (200, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1962-1968
    root.draw(
        &Text::new(
            format!("{}", text_data[2].1),
            (335, 298),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[2]),
            (323, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 1982-1987
    root.draw(
        &Text::new(
            format!("{}", text_data[3].1),
            (458, 280),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[3]),
            (445, 370),
            ("Arial", 18).into_font()
        )
    )?;
    
    // 2004-2009
    root.draw(
        &Text::new(
            format!("{}", text_data[4].1),
            (575, 265),
            ("Arial", 18).into_font()
        )
    )?; 
    root.draw(
        &Text::new(
            format!("{}", candidates[4]),
            (567, 370),
            ("Arial", 18).into_font()
        )
    )?;

    // 2011-2014
    root.draw(
        &Text::new(
            format!("{}", text_data[5].1),
            (697, 313),
            ("Arial", 18).into_font()
        )
    )?;
    root.draw(
        &Text::new(
            format!("{}", candidates[5]),
            (687, 370),
            ("Arial", 18).into_font()
        )
    )?;

    Ok(())
})

Average Number of Ratings for Golden Age Candidates
 
 
0
 
 
100000
 
 
200000
 
 
300000
 
 
400000
 
 
0
 
 
1
 
 
2
 
 
3
 
 
4
 
 
5
 
 
6
 
 
 
 
 
 
 
 
147293
 
 
1945-2020
 
 
436073
 
 
1949-1955
 
 
63041
 
 
1962-1968
 
 
90476
 
 
1982-1987
 
 
112135
 
 
2004-2009
 
 
40125
 
 
2011-2014

As long as you actually looked at the graph before, this comparison shouldn't be too surprising. The lack of books and suvivorship bias more or less explains the early spike.

What suprises me the most though is the steep drop off of 2011-2014. 2004-2009 is clearly buoyed by _The Hunger Games Trilogy_. But I would have expected 2011-2014 to have been equally buoyed by the coming wave of _The Hunger Games Trilogy_ copycats. But upon further reflection and analysis, you can see the two underestimates I made.

First, the most successful inspirees of _The Hunger Games_ came out before 2011. I underestimated how quickly authors would seize upon this small market. Most notably, _The Maze Runner_ by James Dashner and _The Host_ by Stephanie Meyer came out the year after _The Hunger Games_.

Second, I underestimated how quickly the market became saturated. _Divergent_ by Veronica Roth and _The Fifth Wave_ by Rick Yancey were insanely popular book franchises, but they managed to carve out their success from a market overrun with books just like them.

I think this visualization manages to demonstrate that the average number of ratings isn't exactly the key to unlock this question. Showing the total number of ratings is even more flawed. But Goodreads does give us an interesting measurement of popularity, that you may just remember.

In [343]:
// num top 200 / top 10 again
evcxr_figure((800, 400), |root| {
    let top_200 = als::viz::get_number_of_top_200("apocalyptic")?;
    let bottom_190 = als::viz::get_number_of_bottom_190("apocalyptic")?;
    
    let color_one = Palette99::pick(32);
    let color_two = Palette99::pick(31);

    let mut chart = ChartBuilder::on(&root)
        .caption("Number of Most Popular 200 Apocalyptic Books by Year", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .margin_right(20)
        .build_cartesian_2d(1945..2021, 0..25)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_desc("Year")
        .y_desc("Number of Most Popular Books")
        .draw()?;

    let hist_one = Histogram::vertical(&chart)
        .style(color_one.filled())
        .margin(1)
        .data(top_200);
    
    let hist_two = Histogram::vertical(&chart)
        .style(color_two.filled())
        .margin(1)
        .data(bottom_190);
    
    chart.draw_series(hist_one)?
        .label("Top 10")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_one).filled()));
    chart.draw_series(hist_two)?
        .label("Top 200")
        .legend(|(x, y)| Rectangle::new([(x, y - 5), (x + 10, y + 5)], Into::<ShapeStyle>::into(&color_two).filled()));
    
    chart
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    
    Ok(())
})

Number of Most Popular 200 Apocalyptic Books by Year
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Number of Most Popular Books
 
 
Year
 
 
 
 
 
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
 
1950
 
 
 
1960
 
 
 
1970
 
 
 
1980
 
 
 
1990
 
 
 
2000
 
 
 
2010
 
 
 
2020
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Top 10
 
 
Top 200

I told you the Most Popular 200 Books graph would be back. And I also told you it would be less interesting.

Before, I saw this visualization reveal to me the rising and falling popularity of apocalyptic books. Now, all I see is an increasingly rapid march towards total domination. I have no doubt in my mind that 2020 and Climate Change will spawn more apocalyptic fiction than ever before. What I'm less sure of is whether we'll eventually see apocalyptic books come to represent a plurality or even a majority of the Most Popular 200 Books of 2030.

## Synecdoche in Our Golden Ages

### 1949-1955

![Earth Abides](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1566996595l/757009._SY475_.jpg)

_1984_ by George Orwell is probably the obvious choice for the representative book of this potential Golden Age. It's vision of the future has proven so endearing as to possibly have actualized itself into existence. But the sleeper pick for this span is _Earth Abides_ by George R. Stewart from the same year as _1984_, 1949. It is the story of a man surviving a global pandemic only to eventually suffer the far worse horrors of old age. I love _Earth Abides_ as an apocalyptic book because it presents a metaphorical end of the world that we encounter everyday. Ish Williams, the protagonist, manages to survive the end of the world only to feel like his previous world is only truly lost after the younger generations produce a fundamentally alien present to him.

The other facet of _Earth Abides_ that will continue to appear throughout apocalyptic fiction is the reframing of the apocalypse to reflect humanity's more relatable and personal existential fears. To be successful as a piece of apocalyptica, you must balance societal and individual fears. Very few books manage this balance as well as _Earth Abides_.

As I mentioned earlier, there is a real poetry to awarding "The Golden Age of Apocalyptic Literature" to one of it's earliest ages. And as a candidate, 1949-1955 had the highest average weighted rating, the highest number of "apocalyptic" tags, and the most ratings of any other age. But equally and oppositely there exists the urge to turn away from the earliest examples. The ideas originally presented have been refined and refactored into something even crueler and more terrifying. The further away we get from this era the less sympathetic that society's fears. Surely citizens of America during the Cuban Missile Crisis felt a greater fear of doomsday having brushed so close. But equally surely I, still stuck in 2020 watching two old, white men argue about who's going to be doing more fracking, having experienced all previous doomsday fears through literature have a greater anxiety for the apocalypse.

Right?

### 1962-1968

![Cat's Cradle](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1288718321l/386411._SY475_.jpg)

Between the ages of 8-12 I read every Kurt Vonnegut book. I would go on to re-read all of them multiple times throughout my life. (Actually, I should re-read one of them soon. Maybe _The Sirens of Titan_? It's crazy underrated.)

_Cat's Cradle_ by Kurt Vonnegut is easily the funniest book about the end of the world. Published one year after The Cuban Missile Crisis, Kurt Vonnegut would explore our relationship with technology and it's destructive capacity with as much humor as one can have about the closest the world has ever come to truly ending. What I love most about _Cat's Cradle_ is that it not only presents an end of the world that has only aged increasingly well despite it's immediacy and allegorical nature, but it also presents a remedy to the societal ills that Kurt Vonnegut had diagnosed us with: compassion. Kurt Vonnegut saw that the less people cared about other people, the less they cared about what their amoral scientific progress rendered unto the world.

When I initially set about this project, I secretly had a hunch that that the years following the Cuban Missile Crisis would reveal humanity's ultimate dread. 1962-1968 has disappointed me as candidate given all my hopes for it. But as a woefully inadequate and underqualified historian, I don't think humanity has ever been as close to total annihilation as we were that October six decades ago.

Spooky.

(Also, here's where I shout out the best apocalyptic non-fiction of 1962, _Silent Spring_ by Rachel Carson.)

### 1982-1987

![The Handmaid's Tale](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1578028274l/38447._SY475_.jpg)

One of the more diverse and compelling Golden Ages, 1982-1987 is really where we begin to see the rise of other possible ends of the world. (Though I really wanted to give it to _Emergence_ by David R. Palmer just to give it to another plague-based-apocalypse book.) You have to wonder whether the lack of apocalyptic fiction in the previous decade had lowered readers guard. It is definitely _a_ way to explain why 1982-1987 had the highest rate of "apocalyptic" tagging and anomalously high popularity that wouldn't be matched for another 20 years.

_Watchmen_ by Alan Moore depicts one of the most provocative doomsdays ever imagined. Another in the tradition of _Earth Abides_ where the apocalypse is both metaphorical and real, Moore has us question - or, more accurately, successfully casts doubt upon - whether the ends could ever justify the means. And he did this by taking the premise, in true 80s fashion, *to the max*. If the ends were world peace, would any means therefore be justifiable? What about the deaths of millions and possible nuclear armageddon? What does it mean to have power, or to have power vested in you?

_Nausicaä of the Valley of the Wind_ by Hayao Miyazaki is a little bit _Astro Boy_, a little bit _Dune_ (okay, a lot of _Dune_), and a lot of environmentalism. From the time I was scarred as a child by "The Seven Days of Fire" (drawn by Hideki Anno of _Neon Genesis Evangelion_ fame), until I read the manga as a high schooler in the attached Starbucks of a Barnes and Noble in The Plaza of Kansas City, I would never forget the basic message that Miyazaki was trying to communicate, and which rings truer and truer as we destroy the climate: Earth will survive, whether or not humans do.

Clearly something about this Golden Age echoes through the halls of the 21st Century. So much of the apocalyptic film and television from the 80s has persisted to this day. But maybe even more so than _Watchmen_, _The Handmaid's Tale_ by Margaret Atwood would be loved by audiences in its 2010s televised form. Apocalypticism is inherently rooted in religion. Recent musings about Climate Change have assuaged my hostile stance on proselytization. I'm still not a big fan, to be clear, but if you knew the world was ending, wouldn't you tell everyone you could? Especially if there was still time to do something about it.

Every day, Gilead seems realer and closer. Yet here I am, proselytizing.

The world is ending, and we are running out of time to do something about it.

### 2004-2009

![The Road](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1600241424l/6288.jpg)

2004-2009 is the transitional Golden Age. This age would see apocalyptic fiction burst from the pages of _Astounding Science Fiction_ into the mainstream. Despite this, I chose neither of the juggernauts of this age.

_The Road_ by Cormac McCarthy is one of the most depressing books ever written. Full stop. My father recommended the book to me in early high school. At the time, it was so bleak as to be nigh incomprehensible to a 14 year old in suburban Kansas. Re-reading it five years ago, and thinking on it now, I was less saddened by the nihilism of the book, and more the nihilism that must have so overwhelmed my father. _An Inconvenient Truth_ came out the same year as _The Road_. I remember the 2000 election, and I remember my parents being upset that night. I was reminded of this vignette while reading a book about a father trying to get his son to safety in an unsafe and shattered world. The Sisyphean task's analogy of trying to raise a child in a ecologically dwindling world was not lost on me.

Of course though, we can't ignore the elephants in the room. _The Hunger Games_ by Suzanne Collins took familiar distopian tropes from many different sources and then related the struggles of her tween and teen audience to the end of the world. (An extremely popular move with teenagers.) And did so so successfully as to singlehandedly spawn a literary and cinematic leviathan.

_World War Z_ by Max Brooks took the genre experimentation of the 80s Golden Age, sprinkled in a dash of zombies, and laser focused in on the blooming isolation of an increasingly online world. The zombies of _World War Z_ are, unlike the majority of their ilk, scientifically described and understood. The resounding genius of the novel though is to effectively communicate that this understanding does not make the zombies any less of a threat. It does not dull the fear of the end of the world.

### 2011-2014

![Station Eleven](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1451446835l/20170404.jpg)

I have never read _Station Eleven_ by Emily St. John Mandel. I hadn't even heard of it until starting this project. It sure seems apropos: plagues, existentialism, self-importance, etc. Its pertinence may be a hindrance to it as synecdoche though.

_Divergent_ by Veronica Roth and _The Fifth Wave_ by Rick Yancey are two other books I haven't read. They take the extremely successful formula exacted by _The Hunger Games_ and twist the future distopias into even less recognizable forms. These books and their film franchises and their marketing would cause the ballooning of the genre you see represented in almost all of our graphs.

After penning the _Mistborn_ trilogy, Brandon Sanderson would set out to finish one of the most popular fantasy series of all time after its creator died. _The Wheel of Time_ series would share an aspect in common with his _Mistborn_ and _The Stormlight Archive_ series: an obsession with the end of the world. "Saving the world" is a common trope of fantasy, but rarely do you see the armageddon happen. And rarely do you see it as successfully performed and described as in the writings of Brandon Sanderson. The books are highly reviewed after all, as our analysis shows. Maybe this shows that apocalypses are still a fear readers want to engage with, even in the unfamiliar forms they take in fantasy.

Or maybe readers need the distance a fantasy world provides to engage with the idea at all.

## Conclusion

Bertolt Brecht famously said that "Art is not a mirror held up to society but a hammer with which to shape it."

You may have skipped to this section to get the quick answer. I don't blame you. If you think trying to read and understand all of this doesn't seem worth it, try writing it all only to settle on the answer of unanswerability. I no longer believe there is a single Golden Age. It may still be yet to come, or the concept itself may be flawed; trying to divide the modern era into nice boxes finally feels as fruitless as it sounds. The hammer of art is constant and chaotic.

When I first pitched this idea to a friend I also included my hypothesis (that I didn't dare put at the start in order to reduce the amount of science being done here): things seem bleak now, but the authors and audience of the 50s and 60s haven't had 3/4ths of a century to process nuclear anxiety. I sought to prove this hypothesis by finding evidence that apocalyptic fiction had been more prevalent, more peerless, and more popular in the past. I hoped that by showing this to be the case, I could show myself that my fears were exagerrated.

As I head into weekend five of this project, I'm reminded of Cormac McCarthy's process for writing _The Road_. Driving through El Paso, Texas with his son, McCarthy wondered whether the hills in front of him would be on fire in fifty to a hundred years. A few years later he would write the entire book in six weeks in Ireland. Last week, after a long night of trying to generate a 3D manifold for a graph on this project, I found myself watching the final Presidential Debate of 2020. On my phone, in the toilet room off of my bathroom, watching the moderator change the debate subject to Climate Change, I was having an anxiety attack; crying and whispering "You're killing us. You're killing everyone. Don't you get it?" I wonder if Cormac McCarthy felt unburdened after he stopped living with that thought in his head, and had captured it in writing. I hope so.

Maybe you've been convinced that one of the Golden Ages I presented is _the_ Golden Age of Apocalyptic Literature. Maybe you've been convinced that I'm an unmitigated weirdo. Maybe both.

If art is a mirror, then it seems to reflect a society growing ever more aware of it's tenuous grasp on existence. It reflects a society that has been afraid and triumphed in spite of it. It reflects a society that is increasingly willing to engage with the end of the world and learn how to handle and avoid it.

If art is a hammer, then the authors of 1949-1955 shaped a society that could withstand the power it itself had wrought. The authors of 1962-1968 shaped a society that could come to the very brink of total destruction, laugh about it, and be better for it. The authors of 1982-1987 shaped a society that could see the potential devastating abilities of weapons far smaller and far less obvious than the atomic bomb. The authors of 2004-2009 shaped a society more aware of the absurd task it had placed upon the shoulders of those too young to realize its weight. The authors of 2011-2014 shaped a society that lives and breathes doomsday.

The mirror of the years to come may reflect a society still grappling with the same fears it always has. Grappling with mortality. Or perhaps the hammer of art to come will shape a society capable of pushing the boulder all the way to the top of the mountain.

Perosnally, I find myself convinced of the hammer.